Demo of custom functions for MPOD that can be called easily from this notebook: 
- Ramping based on fixed intervals 
- Ramping like a voltage divider

In [1]:
from Driver.MPODClass import MPOD#Basic functionalities for HV Control
from Driver.MPODCustomFunctions import CustomFx #Custom functions for advanced control

Crate = MPOD(IP='169.254.107.70', MIBdir='/usr/share/snmp/mibs')# default settings
FX = CustomFx(Crate, active_modules = [0], channel_names = ['Drift','GEM Top+','GEM Top-','GEM Mid+','GEM Mid-','GEM Low+','GEM Low-','None'])
#These settings are not the default and need to be kept the same for these to work. Order of channels can be changed as long as text names are moved accordingly

#Basic setup validation: 
print('Crate power state: ', Crate.QueryPowerCrate())
print('All connected channels: ', Crate.GetAllNames())
for idx, name in enumerate(FX.channel_names):
    print('Channel ',FX.active_channels[idx], ' is designated as ', name)


Crate power state:  1
All connected channels:  [0, 1, 2, 3, 4, 5, 6, 7, 700, 701, 702, 703, 704, 705, 706, 707, 800, 801, 802, 803, 804, 805, 806, 807]
Channel  0  is designated as  Drift
Channel  1  is designated as  GEM Top+
Channel  2  is designated as  GEM Top-
Channel  3  is designated as  GEM Mid+
Channel  4  is designated as  GEM Mid-
Channel  5  is designated as  GEM Low+
Channel  6  is designated as  GEM Low-
Channel  7  is designated as  None


Now the ramping functions can be demonstrated.
1) Ramp all to zero
2) Ramp like a voltage divider (x2 absolute values, Drift & GEM, with ratios for dividing GEM voltage)
3) Ramp all by a fixed, global dV increment (x1 dV as input)
4) Ramp all with individual dV increments (x7 dV as input) NOT YET IMPLEMENTED
5) Ramp drift to a voltage, then set dV from drift NOT YET IMPLEMENTED


In [2]:
FX.RampAll()#default is to ramp all to zero

Ramp values: [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]


In [9]:
#Voltage divider method: Requires user to review inputs (printed below) before sending by entering SEND
#User input box should appear in text search box at top of window. Enter SEND there! 
FX.RampAll()#Ramps all to zero before next set

V_Drift = 20
V_GEM =  15 #Max V on GEMs
divider_fractions = [1,0.8,0.5,0.3,0.2,0.1]#top to bottom, fractions of V_GEM

print('V  Targets before ramp: ', Crate.GetAllTargetVoltages())
print('V Actuals before ramp: ', Crate.QueryAllVoltages())
FX.RampMethod('Voltage Divider',[20,15,*divider_fractions])
print('V Targets after ramp: ',Crate.GetAllTargetVoltages())
print('V Actuals after ramp: ',Crate.QueryAllVoltages())


Ramp values: [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
V  Targets before ramp:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V Actuals before ramp:  [-0.179520338774, -6.254049777985, -5.365427494049, -3.418344259262, -1.955440044403, -1.295484542847, -2.029622077942, 0.146110400558, -0.082106798887, -0.06276217103, -0.08029948175, -0.036019951105, -0.052145071328, -0.047138385475, -0.058929122984, -0.043098073453, -0.006548100617, 0.022867817432, 0.008925922215, 0.010915733874, 0.017846066505, 0.024348381907, 0.022974584252, 0.000639718026]
Confirm settings below before sending
Drift voltage   on channel 0: 20 V
GEM Top voltage on channel 1 & 2: 15, 12.0 V
GEM Mid voltage on channel 3 & 4: 7.5, 4.5 V
GEM Low voltage on channel 5 & 6: 3.0, 1.5 V
V Targets after ramp:  [20.0, 15.0, 12.0, 7.5, 4.5, 3.0, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V Actuals 

In [ ]:
#Demo of ramping with aproximation of equal ramp rates: 
#TODO: Test this and make edits if needed. likely buggy... 
FX.RampTogether(channels = [0,1], target_voltage = [50,100])
# FX.RampTogether(FX.ActiveChannels,)
# RampTogether(self, channels = None, target_voltage = None

In [2]:
FX.RampAll()#Ramps all to zero before next set

v_to_increment = 50

print('V  Targets before ramp: ', Crate.GetAllTargetVoltages()[0:FX.n_channels])
print('V Actuals before ramp: ', Crate.QueryAllVoltages()[0:FX.n_channels])
FX.RampMethod('Increment All', v_to_increment)#Increments all channels by 50 V
print('V Targets after ramp: ',Crate.GetAllTargetVoltages()[0:FX.n_channels])
print('V Actuals after ramp: ',Crate.QueryAllVoltages()[0:FX.n_channels])


Ramp values: [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
V  Targets before ramp:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V Actuals before ramp:  [-0.985886335373, -1.161875963211, -0.88283598423, -0.627353131771, -0.958697676659, -1.037932395935, -1.785813331604, 0.146110400558]
V Targets after ramp:  [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 0.0]
V Actuals after ramp:  [-50.05827331543, -49.989009857178, -50.033718109131, -49.985118865967, -61.856086730957, -62.255569458008, -61.073413848877, 0.146110400558]


In [4]:
FX.RampAll()#Ramps all to zero before next set

v_to_increment = [0,10,20,30,40,50,60]

print('V  Targets before ramp: ', Crate.GetAllTargetVoltages())
print('V Actuals before ramp: ', Crate.QueryAllVoltages())
FX.RampMethod('Individual Increments', v_to_increment)#Increments each channels by increment value
print('V Targets after ramp: ',Crate.GetAllTargetVoltages())
print('V Actuals after ramp: ',Crate.QueryAllVoltages())


Ramp values: [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
V  Targets before ramp:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V Actuals before ramp:  [-0.611226975918, -4.086909294128, -10.229514122009, -0.780178666115, -21.586030960083, -28.081928253174, -58.671035766602, 0.146110400558, -0.083080738783, -0.051083371043, -0.076410382986, -0.031150132418, -0.05798253417, -0.051027804613, -0.056006200612, -0.042125351727, -0.006548100617, 0.02236925438, 0.009921967983, 0.010416998528, 0.016849221662, 0.022356279194, 0.022974584252, 0.000639718026]
V Targets after ramp:  [0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V Actuals after ramp:  [-0.992053627968, -8.771566390991, -29.45073890686, -42.205429077148, -52.179512023926, -12.433365821838, -22.518489837646, 0.146110400558, -0.080158933997, -0.06276217103, -0.079327210784